# วิเคราะห์เชิงลึก: ข้อมูลการช่วยเหลือน้ำท่วม
## Deep Insights - Flood Relief Data Analysis

**วันที่:** 23-26 พฤศจิกายน 2568 (Bangkok Time UTC+7)  
**สถานที่:** สงขลา (Songkhla Province)  
**แหล่งข้อมูล:** BigQuery Mart Tables  
**ประเภท:** Deep Statistical & Trend Analysis with Insights

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pytz
import warnings
warnings.filterwarnings('ignore')

BKK_TZ = pytz.timezone('Asia/Bangkok')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.2f}'.format)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.family'] = 'TH SarabunPSK'

print('='*80)
print('✓ การตั้งค่าเสร็จสิ้น')
print(f'✓ Timezone: {BKK_TZ} (UTC+7)')
print('='*80)

✓ การตั้งค่าเสร็จสิ้น
✓ Timezone: Asia/Bangkok (UTC+7)


In [ ]:
import matplotlib as mpl
mpl.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
mpl.rc('font', family='Sarabun')

---\n## SECTION 1: โหลดและเตรียมข้อมูล

In [2]:
print('\n📥 กำลังโหลดข้อมูล...\n')

requests_by_date = pd.read_csv('/Users/coraline/Documents/flood_relief/mart_request_by_date_victims.csv')
mart_sos = pd.read_csv('/Users/coraline/Documents/flood_relief/mart_sos.csv')

print(f'✓ mart_request_by_date_victims: {len(requests_by_date):,} records')
print(f'✓ mart_sos: {len(mart_sos):,} records')

requests_by_date['created_date'] = pd.to_datetime(requests_by_date['created_date'])
if requests_by_date['created_date'].dt.tz is None:
    requests_by_date['created_date'] = requests_by_date['created_date'].dt.tz_localize('UTC')
requests_by_date['created_date'] = requests_by_date['created_date'].dt.tz_convert(BKK_TZ)
requests_by_date['created_date_only'] = requests_by_date['created_date'].dt.date

datetime_cols = ['updated_at', 'created_at', 'location_updated_at', 'expired_at']
for col in datetime_cols:
    if col in mart_sos.columns:
        mart_sos[col] = pd.to_datetime(mart_sos[col], errors='coerce')
        if mart_sos[col].dt.tz is None:
            mart_sos[col] = mart_sos[col].dt.tz_localize('UTC')
        mart_sos[col] = mart_sos[col].dt.tz_convert(BKK_TZ)

print('✓ แปลง DateTime สำเร็จ')

victim_categories = ['cnt_general', 'cnt_elderly', 'cnt_child', 'cnt_bedridden', 'cnt_disabled', 'cnt_homebound', 'cnt_pet']
requests_by_date['total_victims'] = requests_by_date[victim_categories].sum(axis=1)
total_all_victims = requests_by_date['total_victims'].sum()
total_locations = len(mart_sos)


📥 กำลังโหลดข้อมูล...

✓ mart_request_by_date_victims: 9 records
✓ mart_sos: 9,186 records
✓ แปลง DateTime สำเร็จ


---\n## SECTION 2: INSIGHT #1 - ความเร่งด่วนของสถานการณ์

In [3]:
print('\n' + '='*80)
print('🚨 INSIGHT #1: ความเร่งด่วนของสถานการณ์ (Urgency Level)')
print('='*80 + '\n')

status_summary = requests_by_date.groupby('status_text').agg({
    'total_victims': 'sum',
    'created_date': 'count'
}).reset_index()
status_summary.columns = ['Status', 'Victims', 'Records']

waiting_help = status_summary[status_summary['Status'] == 'รอการช่วยเหลือ']['Victims'].values[0]
being_helped = status_summary[status_summary['Status'] == 'กำลังดำเนินการช่วยเหลือ...']['Victims'].values[0]
already_helped = status_summary[status_summary['Status'] == 'ช่วยเหลือแล้ว']['Victims'].values[0]

pct_waiting = (waiting_help / total_all_victims) * 100
pct_being = (being_helped / total_all_victims) * 100
pct_done = (already_helped / total_all_victims) * 100

print(f'📊 สรุปสถานะผู้ประสบภัย:\n')
print(f'  🔴 รอการช่วยเหลือ (CRITICAL):         {int(waiting_help):7,} คน ({pct_waiting:5.1f}%)')
print(f'  🟡 กำลังดำเนินการช่วยเหลือ (IN PROGRESS): {int(being_helped):7,} คน ({pct_being:5.1f}%)')
print(f'  🟢 ช่วยเหลือแล้ว (RESOLVED):          {int(already_helped):7,} คน ({pct_done:5.1f}%)')
print(f'  {"─"*60}')
print(f'  รวมทั้งสิ้น:                         {int(total_all_victims):7,} คน (100.0%)')

print(f'\n💡 สำคัญ:')
if pct_waiting > 50:
    print(f'   ⚠️  มีผู้ประสบภัยกว่า {pct_waiting:.1f}% ยังรอการช่วยเหลือ - สถานการณ์เร่งด่วน')
else:
    print(f'   ✓ ดำเนินการช่วยเหลือได้อย่างมีประสิทธิผล')

print('\n' + '='*80)


🚨 INSIGHT #1: ความเร่งด่วนของสถานการณ์ (Urgency Level)

📊 สรุปสถานะผู้ประสบภัย:

  🔴 รอการช่วยเหลือ (CRITICAL):          32,724 คน ( 99.8%)
  🟡 กำลังดำเนินการช่วยเหลือ (IN PROGRESS):      13 คน (  0.0%)
  🟢 ช่วยเหลือแล้ว (RESOLVED):               37 คน (  0.1%)
  ────────────────────────────────────────────────────────────
  รวมทั้งสิ้น:                          32,774 คน (100.0%)

💡 สำคัญ:
   ⚠️  มีผู้ประสบภัยกว่า 99.8% ยังรอการช่วยเหลือ - สถานการณ์เร่งด่วน



---\n## SECTION 3: INSIGHT #2 - กลุ่มเสี่ยง (Vulnerable Groups)

In [4]:
print('\n' + '='*80)
print('⚠️  INSIGHT #2: กลุ่มเสี่ยงพิเศษ (Vulnerable Groups)')
print('='*80 + '\n')

vulnerable_categories = {
    'ผู้สูงอายุ (Elderly)': 'cnt_elderly',
    'เด็ก (Children)': 'cnt_child',
    'ผู้พิการ (Disabled)': 'cnt_disabled',
    'ติดเตียง (Bedridden)': 'cnt_bedridden',
    'เรือนจำ (Homebound)': 'cnt_homebound'
}

vulnerable_data = []
for name, col in vulnerable_categories.items():
    total = requests_by_date[col].sum()
    pct = (total / total_all_victims) * 100 if total_all_victims > 0 else 0
    vulnerable_data.append({'Group': name, 'Count': int(total), 'Percent': pct})

vulnerable_df = pd.DataFrame(vulnerable_data).sort_values('Count', ascending=False)
total_vulnerable = vulnerable_df['Count'].sum()
pct_vulnerable = (total_vulnerable / total_all_victims) * 100

print('👥 จำนวนผู้ประสบภัยในกลุ่มเสี่ยง:\n')
for idx, row in vulnerable_df.iterrows():
    print(f'  {row["Group"]:30} : {row["Count"]:6,} คน ({row["Percent"]:5.1f}%)')

print(f'\n💡 สำคัญ:')
top_vulnerable = vulnerable_df.iloc[0]
print(f'   ⚠️  {top_vulnerable["Group"]} มีจำนวนมากที่สุด: {int(top_vulnerable["Count"]):,} คน')
print(f'   ⚠️  รวมกลุ่มเสี่ยง: {int(total_vulnerable):,} คน ({pct_vulnerable:.1f}% ของผู้ประสบภัยทั้งหมด)')

print('\n' + '='*80)


⚠️  INSIGHT #2: กลุ่มเสี่ยงพิเศษ (Vulnerable Groups)

👥 จำนวนผู้ประสบภัยในกลุ่มเสี่ยง:

  ผู้สูงอายุ (Elderly)           : 10,011 คน ( 30.5%)
  เด็ก (Children)                :  6,669 คน ( 20.3%)
  เรือนจำ (Homebound)            :  1,394 คน (  4.3%)
  ติดเตียง (Bedridden)           :  1,014 คน (  3.1%)
  ผู้พิการ (Disabled)            :    708 คน (  2.2%)

💡 สำคัญ:
   ⚠️  ผู้สูงอายุ (Elderly) มีจำนวนมากที่สุด: 10,011 คน
   ⚠️  รวมกลุ่มเสี่ยง: 19,796 คน (60.4% ของผู้ประสบภัยทั้งหมด)



---\n## SECTION 4: INSIGHT #3 - แนวโน้มเวลา (Timeline Trend)

In [5]:
print('\n' + '='*80)
print('⏰ INSIGHT #3: แนวโน้มเวลาและการเปลี่ยนแปลง (Timeline & Trend)')
print('='*80 + '\n')

daily_data = requests_by_date.groupby('created_date_only')['total_victims'].sum()

print('📊 จำนวนผู้ประสบภัยตามวันที่:\n')
for date, victims in daily_data.items():
    bar = '▓' * int(victims / 500)
    print(f'  {date} : {int(victims):6,} คน {bar}')

if len(daily_data) > 1:
    day1 = daily_data.iloc[0]
    last_day = daily_data.iloc[-1]
    change = last_day - day1
    pct_change = (change / day1) * 100 if day1 > 0 else 0
    
    print(f'\n📈 การเปลี่ยนแปลง:\n')
    print(f'  วันแรก:        {int(day1):6,} คน')
    print(f'  วันสุดท้าย:    {int(last_day):6,} คน')
    print(f'  การเปลี่ยนแปลง: {int(change):6,} คน ({pct_change:+.1f}%)')
    
    if pct_change > 0:
        print(f'\n  📈 เพิ่มขึ้น - สถานการณ์ยังคงรุนแรง')
    elif pct_change < 0:
        print(f'\n  📉 ลดลง - สถานการณ์กำลังดีขึ้น')

max_day = daily_data.idxmax()
max_victims = daily_data.max()
print(f'\n💡 สำคัญ:')
print(f'   • วันที่รุนแรงสุด: {max_day} ({int(max_victims):,} คน)')
print(f'   • เป็น {(max_victims/total_all_victims)*100:.1f}% ของผู้ประสบภัยทั้งหมด')

print('\n' + '='*80)


⏰ INSIGHT #3: แนวโน้มเวลาและการเปลี่ยนแปลง (Timeline & Trend)

📊 จำนวนผู้ประสบภัยตามวันที่:

  2025-11-23 :      4 คน 
  2025-11-24 :  1,542 คน ▓▓▓
  2025-11-25 : 28,580 คน ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
  2025-11-26 :  2,648 คน ▓▓▓▓▓

📈 การเปลี่ยนแปลง:

  วันแรก:             4 คน
  วันสุดท้าย:     2,648 คน
  การเปลี่ยนแปลง:  2,644 คน (+66100.0%)

  📈 เพิ่มขึ้น - สถานการณ์ยังคงรุนแรง

💡 สำคัญ:
   • วันที่รุนแรงสุด: 2025-11-25 (28,580 คน)
   • เป็น 87.2% ของผู้ประสบภัยทั้งหมด



---\n## SECTION 5: INSIGHT #4 - ข้อมูลสถานที่

In [6]:
print('\n' + '='*80)
print('📍 INSIGHT #4: ข้อมูลสถานที่และสถานะทางการแพทย์')
print('='*80 + '\n')

province_stats = mart_sos.groupby('province').agg({
    '_id': 'count',
    'patient': 'sum'
}).reset_index()
province_stats.columns = ['Province', 'Locations', 'Patients']
province_stats = province_stats.sort_values('Locations', ascending=False).head(10)

print('📊 10 จังหวัดที่มีสถานที่มากที่สุด:\n')
print(province_stats.to_string(index=False))

top_province = province_stats.iloc[0]
print(f'\n💡 สำคัญ:')
print(f'   • จังหวัดที่ประสบปัญหามากที่สุด: {top_province["Province"]} ({int(top_province["Locations"]):,} สถานที่)')
print(f'   • มีผู้ป่วยในจังหวัดดังกล่าว: {int(top_province["Patients"]):,} คน')

print('\n' + '='*80)


📍 INSIGHT #4: ข้อมูลสถานที่และสถานะทางการแพทย์

📊 10 จังหวัดที่มีสถานที่มากที่สุด:

     Province  Locations  Patients
        สงขลา       9034      9571
         สตูล         35        42
            -         30        50
         ยะลา         20        20
       พัทลุง         15        20
      ปัตตานี          8        14
กรุงเทพมหานคร          7         7
       ชลบุรี          7         7
         ตรัง          7         7
 สุราษฎร์ธานี          6         6

💡 สำคัญ:
   • จังหวัดที่ประสบปัญหามากที่สุด: สงขลา (9,034 สถานที่)
   • มีผู้ป่วยในจังหวัดดังกล่าว: 9,571 คน



---\n## SECTION 6: INSIGHT #5 - ผู้ป่วยเสี่ยงสูง

In [7]:
print('\n' + '='*80)
print('🚨 INSIGHT #5: ผู้ป่วยเสี่ยงสูง (High-Risk Patients)')
print('='*80 + '\n')

bedridden_count = (mart_sos['is_bedridden'] == True).sum()
elderly_count = (mart_sos['is_elderly'] == True).sum()
chronic_patients = (mart_sos['disease'].str.contains('โรค', na=False)).sum()

print('👨‍⚕️ การวิเคราะห์ความเสี่ยง:\n')
if bedridden_count > 0:
    print(f'  🔴 ผู้ป่วยติดเตียง (Bedridden):  {int(bedridden_count):5,} คน ({(bedridden_count/len(mart_sos))*100:5.1f}%)')
if elderly_count > 0:
    print(f'  🟠 ผู้สูงอายุ (Elderly):         {int(elderly_count):5,} คน ({(elderly_count/len(mart_sos))*100:5.1f}%)')
if chronic_patients > 0:
    print(f'  🟡 ผู้ป่วยโรคประจำตัว (Chronic): {int(chronic_patients):5,} สถานที่ ({(chronic_patients/len(mart_sos))*100:5.1f}%)')

print(f'\n💡 สำคัญ:')
if bedridden_count > 0:
    print(f'   🚨 มีผู้ป่วยติดเตียง {int(bedridden_count):,} คนต้องการการช่วยเหลือเร่งด่วน')
if elderly_count > 0:
    print(f'   ⚠️  มีผู้สูงอายุ {int(elderly_count):,} คนที่ต้องการการดูแลพิเศษ')

print('\n' + '='*80)


🚨 INSIGHT #5: ผู้ป่วยเสี่ยงสูง (High-Risk Patients)

👨‍⚕️ การวิเคราะห์ความเสี่ยง:

  🔴 ผู้ป่วยติดเตียง (Bedridden):    374 คน (  4.1%)
  🟠 ผู้สูงอายุ (Elderly):         4,022 คน ( 43.8%)
  🟡 ผู้ป่วยโรคประจำตัว (Chronic): 6,579 สถานที่ ( 71.6%)

💡 สำคัญ:
   🚨 มีผู้ป่วยติดเตียง 374 คนต้องการการช่วยเหลือเร่งด่วน
   ⚠️  มีผู้สูงอายุ 4,022 คนที่ต้องการการดูแลพิเศษ



---\n## SECTION 7: สรุป KEY INSIGHTS

In [8]:
print('\n' + '='*80)
print('🔑 สรุป KEY INSIGHTS')
print('='*80)

print(f"""
╔══════════════════════════════════════════════════════════════════════════╗
║ #1 ความเร่งด่วนสูง: {pct_waiting:.1f}% ยังรอการช่วยเหลือ                      ║
║ #2 กลุ่มเสี่ยง: {int(total_vulnerable):,} คนต้องการการดูแลพิเศษ ({pct_vulnerable:.1f}%)         ║
║ #3 แนวโน้ม: วันที่ {max_day} เสี่ยงสูงสุด ({int(max_victims):,} คน)           ║
║ #4 สถานที่เสี่ยง: {top_province['Province']} ({int(top_province['Locations']):,} สถานที่)                 ║
║ #5 ผู้ป่วยเสี่ยงสูง: {int(bedridden_count)} ติดเตียง, {int(elderly_count)} ผู้สูงอายุ                ║
╚══════════════════════════════════════════════════════════════════════════╝
""")

print('\n📋 สรุปบรรดาสำคัญด้านการจัดการ')
print('='*80)
print(f"""
💬 สถานการณ์: CRITICAL - ต้องการการดำเนินการเร่งด่วน

🎯 ลำดับความสำคัญ:
  1. ลดจำนวนผู้ประสบภัยรอการช่วยเหลือจาก {pct_waiting:.1f}% เป็น < 20%
  2. จัดสรรทรัพยากรสำหรับผู้ป่วยติดเตียง/เสี่ยงสูง
  3. เพิ่มประสิทธิภาพการตอบสนองในจังหวัด {top_province['Province']}
  4. จัดตั้งศูนย์ช่วยเหลือรองในบริเวณเสี่ยงสูง
  5. ติดตามและรายงานผลเป็นรายชั่วโมง

📊 ตัวชี้วัดการติดตาม:
  • อัตราผู้ประสบภัยรอการช่วยเหลือ (Target: < 10%)
  • อัตราการแก้ไข (Target: > 80%)
  • ความคลุมครองในบริเวณเสี่ยง (Target: 100%)
""")

print('='*80)
print('✅ การวิเคราะห์เชิงลึก สมบูรณ์')
print('='*80)


🔑 สรุป KEY INSIGHTS

╔══════════════════════════════════════════════════════════════════════════╗
║ #1 ความเร่งด่วนสูง: 99.8% ยังรอการช่วยเหลือ                      ║
║ #2 กลุ่มเสี่ยง: 19,796 คนต้องการการดูแลพิเศษ (60.4%)         ║
║ #3 แนวโน้ม: วันที่ 2025-11-25 เสี่ยงสูงสุด (28,580 คน)           ║
║ #4 สถานที่เสี่ยง: สงขลา (9,034 สถานที่)                 ║
║ #5 ผู้ป่วยเสี่ยงสูง: 374 ติดเตียง, 4022 ผู้สูงอายุ                ║
╚══════════════════════════════════════════════════════════════════════════╝


📋 สรุปบรรดาสำคัญด้านการจัดการ

💬 สถานการณ์: CRITICAL - ต้องการการดำเนินการเร่งด่วน

🎯 ลำดับความสำคัญ:
  1. ลดจำนวนผู้ประสบภัยรอการช่วยเหลือจาก 99.8% เป็น < 20%
  2. จัดสรรทรัพยากรสำหรับผู้ป่วยติดเตียง/เสี่ยงสูง
  3. เพิ่มประสิทธิภาพการตอบสนองในจังหวัด สงขลา
  4. จัดตั้งศูนย์ช่วยเหลือรองในบริเวณเสี่ยงสูง
  5. ติดตามและรายงานผลเป็นรายชั่วโมง

📊 ตัวชี้วัดการติดตาม:
  • อัตราผู้ประสบภัยรอการช่วยเหลือ (Target: < 10%)
  • อัตราการแก้ไข (Target: > 80%)
  • ความคลุมครองในบริเวณเสี่ยง (Target: